# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb69438d970>
├── 'a' --> tensor([[ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075]])
└── 'x' --> <FastTreeValue 0x7fb69438df70>
    └── 'c' --> tensor([[-0.8731, -0.0325,  0.0672, -0.2724],
                        [ 0.1208,  0.9656, -1.3046,  0.4063],
                        [ 1.7183,  0.5633,  0.7180, -0.8701]])

In [4]:
t.a

tensor([[ 0.1852, -0.8165,  1.7311],
        [-0.2185,  0.4917,  1.3075]])

In [5]:
%timeit t.a

59.4 ns ± 0.0686 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb69438d970>
├── 'a' --> tensor([[ 0.2646, -0.4243,  0.8131],
│                   [ 1.4082, -0.7723,  0.6814]])
└── 'x' --> <FastTreeValue 0x7fb69438df70>
    └── 'c' --> tensor([[-0.8731, -0.0325,  0.0672, -0.2724],
                        [ 0.1208,  0.9656, -1.3046,  0.4063],
                        [ 1.7183,  0.5633,  0.7180, -0.8701]])

In [7]:
%timeit t.a = new_value

60 ns ± 0.0428 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1852, -0.8165,  1.7311],
               [-0.2185,  0.4917,  1.3075]]),
    x: Batch(
           c: tensor([[-0.8731, -0.0325,  0.0672, -0.2724],
                      [ 0.1208,  0.9656, -1.3046,  0.4063],
                      [ 1.7183,  0.5633,  0.7180, -0.8701]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1852, -0.8165,  1.7311],
        [-0.2185,  0.4917,  1.3075]])

In [11]:
%timeit b.a

51.7 ns ± 0.0658 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7711,  0.4830,  0.0806],
               [-1.5910,  0.6893,  0.9972]]),
    x: Batch(
           c: tensor([[-0.8731, -0.0325,  0.0672, -0.2724],
                      [ 0.1208,  0.9656, -1.3046,  0.4063],
                      [ 1.7183,  0.5633,  0.7180, -0.8701]]),
       ),
)

In [13]:
%timeit b.a = new_value

523 ns ± 0.0857 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

864 ns ± 0.225 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 40.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 339 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 948 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb5e8fe0880>
├── 'a' --> tensor([[[ 0.1852, -0.8165,  1.7311],
│                    [-0.2185,  0.4917,  1.3075]],
│           
│                   [[ 0.1852, -0.8165,  1.7311],
│                    [-0.2185,  0.4917,  1.3075]],
│           
│                   [[ 0.1852, -0.8165,  1.7311],
│                    [-0.2185,  0.4917,  1.3075]],
│           
│                   [[ 0.1852, -0.8165,  1.7311],
│                    [-0.2185,  0.4917,  1.3075]],
│           
│                   [[ 0.1852, -0.8165,  1.7311],
│                    [-0.2185,  0.4917,  1.3075]],
│           
│                   [[ 0.1852, -0.8165,  1.7311],
│                    [-0.2185,  0.4917,  1.3075]],
│           
│                   [[ 0.1852, -0.8165,  1.7311],
│                    [-0.2185,  0.4917,  1.3075]],
│           
│                   [[ 0.1852, -0.8165,  1.7311],
│                    [-0.2185,  0.4917,  1.3075]]])
└── 'x' --> <FastTreeValue 0x7fb5e8fe0160>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb5e8fe0700>
├── 'a' --> tensor([[ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075],
│                   [ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075],
│                   [ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075],
│                   [ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075],
│                   [ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075],
│                   [ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075],
│                   [ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075],
│                   [ 0.1852, -0.8165,  1.7311],
│                   [-0.2185,  0.4917,  1.3075]])
└── 'x' --> <FastTreeValue 0x7fb5e8fe0a30>
    └── 'c' --> tensor([[-0.8731, -0.0325,  0.0672, -0.2724],
                        [ 0.1208,  0.9656, -1.3046,  0.4063],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 41.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.8731, -0.0325,  0.0672, -0.2724],
                       [ 0.1208,  0.9656, -1.3046,  0.4063],
                       [ 1.7183,  0.5633,  0.7180, -0.8701]],
              
                      [[-0.8731, -0.0325,  0.0672, -0.2724],
                       [ 0.1208,  0.9656, -1.3046,  0.4063],
                       [ 1.7183,  0.5633,  0.7180, -0.8701]],
              
                      [[-0.8731, -0.0325,  0.0672, -0.2724],
                       [ 0.1208,  0.9656, -1.3046,  0.4063],
                       [ 1.7183,  0.5633,  0.7180, -0.8701]],
              
                      [[-0.8731, -0.0325,  0.0672, -0.2724],
                       [ 0.1208,  0.9656, -1.3046,  0.4063],
                       [ 1.7183,  0.5633,  0.7180, -0.8701]],
              
                      [[-0.8731, -0.0325,  0.0672, -0.2724],
                       [ 0.1208,  0.9656, -1.3046,  0.4063],
                       [ 1.7183,  0.5633,  0.7180, -0.8701]],

In [26]:
%timeit Batch.stack(batches)

81.8 µs ± 89.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.8731, -0.0325,  0.0672, -0.2724],
                      [ 0.1208,  0.9656, -1.3046,  0.4063],
                      [ 1.7183,  0.5633,  0.7180, -0.8701],
                      [-0.8731, -0.0325,  0.0672, -0.2724],
                      [ 0.1208,  0.9656, -1.3046,  0.4063],
                      [ 1.7183,  0.5633,  0.7180, -0.8701],
                      [-0.8731, -0.0325,  0.0672, -0.2724],
                      [ 0.1208,  0.9656, -1.3046,  0.4063],
                      [ 1.7183,  0.5633,  0.7180, -0.8701],
                      [-0.8731, -0.0325,  0.0672, -0.2724],
                      [ 0.1208,  0.9656, -1.3046,  0.4063],
                      [ 1.7183,  0.5633,  0.7180, -0.8701],
                      [-0.8731, -0.0325,  0.0672, -0.2724],
                      [ 0.1208,  0.9656, -1.3046,  0.4063],
                      [ 1.7183,  0.5633,  0.7180, -0.8701],
                      [-0.8731, -0.0325,  0.0672, -0.2724],
                   

In [28]:
%timeit Batch.cat(batches)

150 µs ± 68 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 2.08 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
